In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import json
import numpy

In [ ]:
CHROMEDRIVER_PATH = '/usr/local/bin/chromedriver'
WINDOW_SIZE = "1920,1080"

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,options=chrome_options)

In [ ]:
driver.get('https://delasheshstha.contentder.com/About')

In [ ]:
rows = driver.find_elements_by_class_name('cRow')

In [ ]:
cRows = []
advance_component = ['accordion', 'body', 'bar chart', 'carousel', 'search', 'search result', 'contact us', 'dynamic_cmp_detail', 'dynamic_cmp_list', 'google map', 'iFrame', 'image slider', 'imagebanner', 'line chart', 'login status', 'logo slider', 'managepages', 'menu', 'menupages', 'milestone', 'pie chart', 'Policy Info', 'row separator', 'share page', 'user registration', 'siteheader', 'siteheadermenu', 'skill bar', 'SoundCloudPlayer', 'subscribe', 'forgot password', 'login form', 'password recover', '', 'tabs', 'Text editor', 'unsubscribe form', 'verticalcollector', 'verticalcollectors', 'videobanner', 'layer slider', 'dynamic form']
for row in rows:
    row_data_type = row.get_attribute('data-type')
    if row_data_type not in advance_component:
        cRows.append(row)


In [ ]:
def row_cols(row):
    cCols = []
    selector = row
    editor_row_container = selector.find_elements_by_class_name(
        'editor-row-container')
    if(len(editor_row_container) > 1):
        selector = editor_row_container
    editor_row_shaded_layer = selector.find_elements_by_class_name(
        'editor-row-shaded-layer')
    if(len(editor_row_shaded_layer) > 1):
        selector = editor_row_shaded_layer
    cGrid = selector.find_element_by_css_selector('.cGrid')
    cGrid_divs = decendent(cGrid)
    for cGrid_div in cGrid_divs:
        if(checkClass(cGrid_div.get_attribute('class'), 'cCol')):
            cCols.append(cGrid_div)
    return cCols

In [ ]:
def decendent(selector):
    node = selector.find_elements_by_xpath("*")
    return node

In [ ]:
def checkClass(classes, checkClass):
    return classes.find(checkClass) != -1

In [ ]:
def col_traverse(col):
    structure = [] # Holds inner structure of the element
    components = [] # Holds inner components of the element
    nodes = [] # Holds immediate child nodes of the element

    # Get all immediate child nodes of the given element
    childs = decendent(col)

    for child in childs:
        current_node = {}

        # Get the class of the element
        child_class = child.get_attribute('class')

        # Check if the node element has "editor-component" or "cCol" class
        if(checkClass(child_class, 'editor-component') or checkClass(child_class, 'cCol')):
            data_type = child.get_attribute('data-type')
            nodes.append(data_type)
            components.append(data_type)
            current_node['type'] = data_type
        else:
            current_node['type'] = "unknown"
        
        # Check for inner node
        r_structure, r_components , r_nodes = col_traverse(child)
        components.extend(r_components)
        current_node['components'] = r_components
        current_node['nodes'] = r_nodes
        current_node['child'] = r_structure
        
        # Check if the current_node dict is empty or not
        if bool(current_node):
            # Check of the node is of unknown data type
            if(current_node['type'] == "unknown"): # Get the inner data from the dict
                structure = current_node['child']
                nodes = current_node['nodes']
            else:
                structure.append(current_node)
        else: # If the dict is empty append None instead of empty dict
            structure.append(None)
            
    return structure, components, nodes

In [ ]:
for cRow in cRows:
    row_dict = {}
    row_components = []
    row_nodes = []
    row_structure = []
    # Row Title is not scraped in structure 
    cCols = row_cols(cRow)
    for cCol in cCols:
        col_data_type = cCol.get_attribute('data-type')
        # Traverse the column        
        structure, components, nodes = col_traverse(cCol)

        # Add the col data to row
        row_components.append(col_data_type)
        row_components.extend(components)
        row_nodes.append(col_data_type)

        # Create dict for column
        col_dict = {}
        col_dict['type'] = col_data_type
        col_dict['components'] = components
        col_dict['nodes'] = nodes
        col_dict['child'] = structure

        # Append column to row's structure
        row_structure.append(col_dict)

    components_list = numpy.array(row_components)
    component_names, component_counts = numpy.unique(components_list, return_counts=True)
    component_names = component_names.tolist()
    component_counts = component_counts.tolist()
    counts = []
    for component_name in component_names:
        counts.append("{0}:{1}".format(component_name, component_counts[component_names.index(component_name)]))

    # Create dict for row
    row_dict['type'] = cRow.get_attribute('data-type')
    row_dict['components'] = row_components
    row_dict['count'] = counts
    row_dict['nodes'] = row_nodes
    row_dict['child'] = row_structure
    
    # Pretty print row object
    # print(json.dumps(row_dict, indent=2))
    